In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

# !nbdev_build_lib

In [3]:
inp = pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=5000)

In [4]:
df = next(inp)
df = df[['id',
        'product_name',
        '_keywords' ,
        'ingredients_text',
        'categories',
        'food_groups',
        'nutriments'
        ]]
df = df.rename(columns = {'_keywords':'keywords'}).drop_duplicates('id').set_index('id')
df = df [df['product_name'].str.strip()!=''].dropna(subset = ['product_name'])
df['keywords'] = df['keywords'].apply(lambda x:', '.join(x))
nutriments = pd.json_normalize(df[['nutriments']].to_dict('records')).set_index(df.index)
nutriments.columns = [c.replace('nutriments.','').replace('-','_') for c in nutriments.columns]
df = df.join(nutriments[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']]).drop(columns = ['nutriments']).reset_index()
df['food_groups'] = df['food_groups'].str.replace('en:','')
df = df.dropna(subset = ['energy_kcal_100g'])
insert_ignore(df,'foods_big',unique_cols=['id'])


In [5]:
bigintmax = 2**63-1

In [12]:
for df in tqdm(pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=50000), desc="openfood parsing"):
    df = df[['id',
        'product_name',
        '_keywords' ,
        'ingredients_text',
        'categories',
        'food_groups',
        'nutriments'
        ]]
    df = df[df['id']<bigintmax]
    df = df.rename(columns = {'_keywords':'keywords'}).drop_duplicates('id').set_index('id')
    df = df [df['product_name'].str.strip()!=''].dropna(subset = ['product_name'])
    df['keywords'] = df['keywords'].apply(lambda x:', '.join(x))
    nutriments = pd.json_normalize(df[['nutriments']].to_dict('records')).set_index(df.index)
    nutriments.columns = [c.replace('nutriments.','').replace('-','_') for c in nutriments.columns]
    df = df.join(nutriments[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']]).drop(columns = ['nutriments']).reset_index()
    df['food_groups'] = df['food_groups'].str.replace('en:','')
    df = df.dropna(subset = ['energy_kcal_100g'])
    insert_ignore(df,'foods_big',unique_cols=['id'])





openfood parsing: 6it [10:00, 126.58s/it]

In [ ]:
for df in tqdm(pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=50000), desc="openfood parsing"):